# Functools
Functiones de alto orden, que actuan o retornan funciones, cualquier
objeto callable es tratado como una función para efectos de esto

In [2]:
import functools


In [15]:
student_data = [
    {
        "name": "John Smith",
        "age": 10,
        "on_vacation": False,
        "test_scores": [66, 85, 39, 61, 16, 92, 33, 3, 87, 71],
    },
    {
        "name": "Jane Doe",
        "age": 10,
        "on_vacation": False,
        "test_scores": [4, 73, 75, 4, 50, 83, 8, 23, 42, 23],
    },
    {
        "name": "Isaac Newton",
        "age": 30,
        "on_vacation": True,
        "test_scores": [93, 96, 94, 92, 95, 90, 100, 98, 90, 94],
    },
]

#### Iru_cache, cache
Crea un cache para mejorar velocidad

In [5]:
def factorial(n):
    return n * factorial(n-1) if n else 1

In [6]:
%%timeit
factorial(100)

12.8 µs ± 81.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
@functools.lru_cache(maxsize=None)
def fast_factorial(n):
    return n * factorial(n-1) if n else 1    

In [9]:
%%timeit
fast_factorial(100)

61.5 ns ± 4.66 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
@functools.cache
def fast_factorial(n):
    return n * factorial(n-1) if n else 1

In [11]:
%%timeit
fast_factorial(100)

59.6 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


#### total ordering

In [13]:
from typing import List
class Student:

    def __init__(self, name:str, age:int,test_scores:List[int], on_vacation: bool=False):
        self.name = name
        self.age = age
        self.test_scores = test_scores
        self.on_vacation = on_vacation

    def __repr__(self):
        return f'Student(name={self.name})'

    @property
    def mean_test_score(self):
        return sum(x for x in self.test_scores) / len(self.test_scores)

In [16]:
john = Student(**student_data[0])
newton = Student(**student_data[2])

john, newton

(Student(name=John Smith), Student(name=Isaac Newton))

In [17]:
#esto va a dar error
john < newton

TypeError: '<' not supported between instances of 'Student' and 'Student'

In [18]:
class Student:

    def __init__(self, name:str, age:int,test_scores:List[int], on_vacation: bool=False):
        self.name = name
        self.age = age
        self.test_scores = test_scores
        self.on_vacation = on_vacation
    
    def __repr__(self):
        return f'Student(name={self.name})'

    @property
    def mean_test_score(self):
        return sum(x for x in self.test_scores) / len(self.test_scores)
    
    # add all these special methods.
        
    def __lt__(self, other: Student):
        return self.mean_test_score < other.mean_test_score

    def __le__(self, other: Student):
        return self.mean_test_score <= other.mean_test_score
    
    def __gt__(self, other: Student):
        return self.mean_test_score > other.mean_test_score

    def __ge__(self, other: Student):
        return self.mean_test_score >= other.mean_test_score

    def __eq__(self, other: Student):
        return self.mean_test_score == other.mean_test_score

In [19]:
john = Student(**student_data[0])
newton = Student(**student_data[2])

john, newton

(Student(name=John Smith), Student(name=Isaac Newton))

In [20]:
john > newton

False

In [21]:
# Lo mismo
john.__gt__(newton)

False

In [22]:
functools.total_ordering
class Student:
    
    def __init__(self, name:str, age:int,test_scores:List[int], on_vacation: bool=False):
        self.name = name
        self.age = age
        self.test_scores = test_scores
        self.on_vacation = on_vacation
    
    def __repr__(self):
        return f'Student(name={self.name})'

    @property
    def mean_test_score(self):
        return sum(x for x in self.test_scores) / len(self.test_scores)
    
    # add all these special methods.
        
    def __lt__(self, other: Student):
        return self.mean_test_score < other.mean_test_score

    def __eq__(self, other: Student):
        return self.mean_test_score == other.mean_test_score

#### partial

In [24]:
def is_pass(student: Student, pass_mark = 60):
    passed = student.mean_test_score > pass_mark
    print((f'{student.name} has test score '
          f"{'above' if passed else 'below'} {pass_mark}"))
    return passed

In [25]:
is_pass(john)


John Smith has test score below 60


False

In [26]:
is_pass(newton)


Isaac Newton has test score above 60


True

In [27]:
def is_top_set(student: Student):
    return is_pass(student, 30)

In [28]:
is_top_set(newton)


Isaac Newton has test score above 30


True

In [29]:
is_top_set = functools.partial(is_pass, pass_mark=80)


In [30]:
is_top_set(newton)


Isaac Newton has test score above 80


True

In [31]:
from statistics import median


In [32]:
min([(1, 2), (5, 1), (2, 3)])


(1, 2)

In [33]:
min([(1, 2), (5, 1), (2, 3)], key=lambda item: item[1])


(5, 1)

In [36]:
min_student = functools.partial(min, key=lambda student : median(student.test_scores))
#fin